In [1]:
#Using Overview for Webscraping with OpenAI API from here https://medium.com/@joelpantoja/web-scraping-with-openai-6ceace410191 

In [2]:
import requests
from bs4 import BeautifulSoup
import tiktoken
from requests import Response
from openai import OpenAI, ChatCompletion
from dotenv import load_dotenv
import os

from pprint import pprint

In [3]:
#Load environment and API Key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
# The below function was written with assistance by Gemini. 

import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time

def get_jooble_jobs(search_url, target_count=30):
    driver = uc.Chrome(headless=False) # Keep False initially to monitor
    all_jobs_html = []
    current_page = 1
    
    try:
        while len(all_jobs_html) < target_count:
            # Construct the paginated URL
            paginated_url = f"{search_url}&p={current_page}"
            print(f"Fetching page {current_page}...")
            
            driver.get(paginated_url)
            time.sleep(7) # Wait for Cloudflare and content to settle
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Jooble jobs are usually inside <article> tags or divs with specific classes
            # You want to find the individual containers for each job
            job_cards = soup.find_all('div', class_='+n4WEb rHG1ci')  # Adjust this selector based on actual HTML structure
            
            if not job_cards:
                print("No more jobs found or blocked.")
                break
                
            for card in job_cards:
                if len(all_jobs_html) < target_count:
                    all_jobs_html.append(str(card))
            
            print(f"Collected {len(all_jobs_html)} jobs so far...")
            current_page += 1
            
    finally:
        driver.quit()
        
    return all_jobs_html

base_url = 'https://jooble.org/SearchResult?rgns=Remote&ukw=data%20scientist'
list_of_job_html = get_jooble_jobs(base_url, 30)

Fetching page 1...
Collected 20 jobs so far...
Fetching page 2...
Collected 30 jobs so far...


In [9]:
list_of_job_html

['<div class="+n4WEb rHG1ci" data-test-name="_jobCard" id="5968186734884599128"><div class="_9q4Aij"><h2 class="jA9gFS dUatPc"><a class="_8w9Ce2 tUC4Fj _6i4Nb0 wtCvxI job_card_link" href="https://jooble.org/away/5968186734884599128?p=1&amp;pos=1&amp;rgn=55157&amp;pageType=20&amp;ckey=data+scientist&amp;sid=-8770936937037823462&amp;jobAge=125&amp;relb=10&amp;brelb=10&amp;scr=19999.82367188168&amp;bscr=19999.82367188168&amp;elckey=2547520272143778666&amp;searchTestGroup=1_2_1&amp;iid=8618376717097665519" rel="noopener nofollow" target="_blank">Data Administrator - Research Scientist 2</a></h2><div class="_5A-eEY"><div class="Bdpujj"><button aria-label="Save the vacancy to favorites" class="ZapzXe _9WpygY XEfYdx JFkX+L vMdXr7 RNbuqd" data-test-name="jobFavoritesButton__5968186734884599128" type="button"><span class="H2e2t1 +gVFyQ"><svg class="KaSFrq JAzKjj _0ng4na"><use xlink:href="/assets/sprite.3Zhuck2Rs3KdP9bfefF8.svg#icon-heart_outline-usage"></use></svg></span></button></div></div></

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str = "gpt-3.5-turbo") -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def chat_completion_request(messages, tools=None, tool_choice=None, model="gpt-3.5-turbo"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
            response_format={'type':'json_object'}
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        return e

In [ ]:
# Set OpenAI Format
tools = [
    {
        "type": "function",
        "function": {
            "name": "product_catalog_overview",
            "description": "Retrieves a high level overview of the books in a book catalog page",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "The title of the book",
                    },
                    "price": {
                        "type": "string",
                        "description": "The price of the book in pounds",
                    },
                    "rating": {
                        "type": "integer",
                        "description": "The rating of the book. This should be displayed as a number of stars",
                    },
                    "in_stock": {
                        "type": "boolean",
                        "description": "Whether the book is in stock. The value should be true or false",
                    }
                },
                "required": ["title", "price", "rating"]
            },
        }
    },
]